# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [40]:
# Webdriver: Chrome -> chromedriver
# Webdriver: Firefox -> geckodriver

In [41]:

from selenium import webdriver#Criar navegador
from selenium.webdriver.common.by import By #Localizar elementos (itens de um site)
from selenium.webdriver.common.keys import Keys #permite clicar as teclas do teclado


navegador = webdriver.Chrome()

# Passo 1: Entrar no google
navegador.get('https://www.google.com/')

# Passo 2: Pesquisar a cotação do dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')#botão direito: 'copy Xpath'

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Passo 3: Pegara a cotação do dolar
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# Passo 4: Pesquisar a cotação do euro
navegador.get('https://www.google.com/')

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')#botão direito: 'copy Xpath'

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegar a cotação do dolar
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# Passo 5: Pesquisar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)



5.0916
5.2974
296.86


In [42]:
#!pip install selenium

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [43]:
# Passo 6: Atualizar a minha base de dados com as novas cotações 
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [44]:
# Atualizar a cotação de acordo com a moeda correspondente
#Dolar
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)

#Euro
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)

#Ouro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#display(tabela)
# Atualizar preço de compra = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Atualizar preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.0916,5091.549084,1.40,7128.168718
1,Carro Renault,4500.00,Euro,5.2974,23838.300000,2.00,47676.600000
2,Notebook Dell,899.99,Dólar,5.0916,4582.389084,1.70,7790.061443
3,IPhone,799.00,Dólar,5.0916,4068.188400,1.70,6915.920280
4,Carro Fiat,3000.00,Euro,5.2974,15892.200000,1.90,30195.180000
5,Celular Xiaomi,480.48,Dólar,5.0916,2446.411968,2.00,4892.823936
6,Joia 20g,20.00,Ouro,296.8600,5937.200000,1.15,6827.780000


### Agora vamos exportar a nova base de preços atualizada

In [45]:
tabela.to_excel('Produdos novos.xlsx', index=False)
navegador.quit()